In [16]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sla

In [ ]:
# define lattice and generate hopping matrix with OBC

In [23]:
from pythtb import *
def Tmatrix(t,mu):

    # set model parameters
    mu=0.0
    t=-1.0
    lat=[[1.0,0.0],[0.0,1.0]]
    orb=[[0.0,0.0]]

    # create TB model
    my_model=tb_model(2,2,lat,orb)
    
    # set on-site energies
    my_model.set_onsite([-mu])
    # set hopping terms (one for each nearest neighbor)
    my_model.set_hop(t, 0, 0, [1, 0])
    my_model.set_hop(t, 0, 0, [0, 1])

    # create a 4x4 lattice
    Nx, Ny = 4, 4
    my_ham = my_model.cut_piece(Nx,0,glue_edgs=False)
    my_ham = my_ham.cut_piece(Ny,1,glue_edgs=False)
    # print Hamiltonian
    return my_ham._gen_ham()


In [ ]:
L = 4 * 4
Ntau = 20
dtau = 0.1
U = 4.0
t = -1.0
mu = U/2
beta = dtau * Ntau
alpha = np.arccosh(np.exp(0.5*dtau*U))
T = Tmatrix(t,mu)
ExpT = sla.expm(-dtau * T)
Vdiag = alpha * np.array([[1,-1],]*L)

In [3]:
def ExpM(phi,ExpT,Vdiag):
    # T = exp(-T*dtau)
    return ExpT.dot(np.diag(phi * Vdiag))

In [25]:
def SVD_stablizer(A,B):
    # A * B = A * U * D * V = U1 * D1 * V1 * V = U1 * D1 * V2
    U, D, V = sla.svd(B)
    U, D, V1 = sla.svd(A.dot(U).dot(D))
    return U, D, V1.dot(V)

In [ ]:
def Stable_Bmatrix(phi,ExpT,Vdiag):
    # use SVD algorithm to calculate the stable B matrix
    # ExpV: N x L matrix with each column being exp(- alpha * phi[:,l] * sigma)
    # ExpT: the matrix of exp(-dtau * T)
    for i in range(phi.shape[1]):
        U,D,V = sla.svd(ExpM(phi[:,i],ExpT,Vdiag))

    return U,D,V

In [ ]:
def sweep():
    # start from back sweep and then forward sweep